In [83]:
from typing import Tuple, Dict

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam import PCollection, pvalue, GroupByKey, Map
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

In [84]:
p = beam.Pipeline(InteractiveRunner())

In [85]:
_input: PCollection[Tuple[str, int]] = p | "Create input" >> beam.Create([('a', 1), ('b', 2), ('b', 3), ('c', 4)])
ib.collect(_input)

,0,1
0,a,1
1,b,2
2,b,3
3,c,4


In [86]:
multipliers = p | "Create multipliers" >> beam.Create([('b', 10), ('c', 100)])
ib.collect(multipliers)

,0,1
0,b,10
1,c,100


In [87]:
DEFAULT_MULTIPLIER = 1

In [88]:
def broadcast_join(element: Tuple[str, int], side_input: Dict[str, int]):
    key = element[0]
    val = element[1]
    _multiplier = side_input.get(key, DEFAULT_MULTIPLIER)
    print(key, val, side_input, _multiplier)
    return key, val * _multiplier

joined: PCollection[Tuple[str, int]] = _input | Map(broadcast_join, side_input=pvalue.AsDict(multipliers))
ib.collect(joined)

a 1 {'b': 10, 'c': 100} 1
b 2 {'b': 10, 'c': 100} 10
b 3 {'b': 10, 'c': 100} 10
c 4 {'b': 10, 'c': 100} 100


,0,1
0,a,1
1,b,20
2,b,30
3,c,400


In [89]:
result = joined | GroupByKey() | Map(lambda t: (t[0], sum(t[1])))
ib.collect(result)

,0,1
0,a,1
1,b,50
2,c,400
